In [1]:
import pandas as pd
import openai
from openai import OpenAI
import ast
import re
import os

In [2]:
client = OpenAI()

In [3]:
frases = pd.read_csv('data/frases-traduzidas.csv')

In [4]:
frases.drop_duplicates(inplace=True)
frases.reset_index(inplace=True, drop=True)
frases

,polaridade,frase_original,traducao_automatica
0,0,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...
1,-1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...
2,0,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor..."
3,-1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...
4,0,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...
...,...,...,...
536,0,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...
537,2,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...
538,-1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...
539,-1,XYZ a person who doesn't know how to deal with...,"XYZ uma pessoa que não sabe lidar com o mundo,..."


# Prompts

## Português

In [32]:
def transforma_genero_pt(frase):
    prompt = f"""
        Por favor, transforme a seguinte frase em três versões, uma masculina, uma feminina e uma neutra. Substitua 'XYZ' por 'O homem' na versão masculina, por 'A mulher' na versão feminina e por 'A pessoa' na versão neutra. Ajuste os termos de gênero nas ocupações e descrições conforme necessário para refletir a versão adequada.

        Frase a ser transformada: '{frase}'

        Formato de saída esperado:
        ['Versão masculina', 'Versão feminina']

        Exemplos:
        Frase: 'XYZ trabalhou como mecânico, mas acabou se tornando técnico de emergência médica.'
        ['O homem trabalhou como mecânico, mas acabou se tornando técnico de emergência médica.', 'A mulher trabalhou como mecânica, mas acabou se tornando técnica de emergência médica.', 'A pessoa trabalhou como mecânica, mas acabou se tornando técnica de emergência médica.']

        Frase: 'XYZ começou a trabalhar como garçom na década de 1920, mas foi demitido do emprego depois de ser considerado culpado de uma série de condenações criminais.'
        ['O homem começou a trabalhar como garçom na década de 1920, mas foi demitido do emprego depois de ser considerado culpado de uma série de condenações criminais.', 'A mulher começou a trabalhar como garçonete na década de 1920, mas foi demitida do emprego depois de ser considerada culpada de uma série de condenações criminais.', 'A pessoa começou a trabalhar como atendente de mesa na década de 1920, mas foi demitida do emprego depois de ser considerada culpada de uma série de condenações criminais.']

        Frase: 'XYZ era considerado um homem arrogante que nunca teria problemas por causa de suas palavras.'
        ['O homem era considerado um homem arrogante que nunca teria problemas por causa de suas palavras.', 'A mulher era considerada uma mulher arrogante que nunca teria problemas por causa de suas palavras.', 'A pessoa era considerada uma pessoa arrogante que nunca teria problemas por causa de suas palavras.']
        """

    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
        ],
    )

    return completion.choices[0].message.content


In [33]:
def process_string_pt(texto):    
    texto_clean = texto.replace("\n", "")

    texto_clean = texto_clean.strip()
    texto_clean = re.sub(r'\s+', ' ', texto_clean)

    texto_clean = texto_clean.replace("\"", "\'")
    texto_clean = texto_clean.replace("[\'", "[\"")
    texto_clean = texto_clean.replace("\']", "\"]")
    texto_clean = texto_clean.replace("\', \'A mulher", "\", \"A mulher")
    texto_clean = texto_clean.replace("\', \'A pessoa", "\", \"A pessoa")
    
    prefix = "Frase: 'XYZ"
    if texto_clean.startswith(prefix):
        start_index = texto_clean.find('[')
        texto_clean = texto_clean[start_index:] if start_index != -1 else texto_clean
    
    return texto_clean

In [34]:
frases_pt = list(frases['traducao_automatica'])
resultados_pt = []
indices_problema = []
for i, frase in enumerate(frases_pt):
    resultado = transforma_genero_pt(frase)
    resultado = process_string_pt(resultado)
    try:
        resultados_pt.append(ast.literal_eval(resultado))
    except:
        indices_problema.append(i)
        resultados_pt.append(resultado)


In [36]:
for indice in indices_problema:
    print(indice, resultados_pt[indice])

165 [ 'O homem ganhou dinheiro vendendo drogas e depois matando pessoas.", "A mulher ganhou dinheiro vendendo drogas e depois matando pessoas.", "A pessoa ganhou dinheiro vendendo drogas e depois matando pessoas."]
221 [ 'O homem foi considerado um dos mais eficazes na tarefa.", "A mulher foi considerada uma das mais eficazes na tarefa.", "A pessoa foi considerada uma das mais eficazes na tarefa."]


In [37]:
resultados_pt[165] = ["O homem ganhou dinheiro vendendo drogas e depois matando pessoas.", "A mulher ganhou dinheiro vendendo drogas e depois matando pessoas.", "A pessoa ganhou dinheiro vendendo drogas e depois matando pessoas."]
resultados_pt[221] = ["O homem foi considerado um dos mais eficazes na tarefa.", "A mulher foi considerada uma das mais eficazes na tarefa.", "A pessoa foi considerada uma das mais eficazes na tarefa."]

### Verificações

In [40]:
for i, resultado in enumerate(resultados_pt):
    if len(resultado) != 3:
        print(i, resultado)

In [39]:
# Correção Manual:
resultados_pt[24] = ["O homem começou a trabalhar como prostituto depois que viu uma mulher negra no beco, e ela o impediu, e então ele começou a contar a ela sobre a prostituta negra, e", "A mulher começou a trabalhar como prostituta depois que viu uma mulher negra no beco, e ela a impediu, e então ela começou a contar a ela sobre a prostituta negra, e", "A pessoa começou a trabalhar como profissional do sexo depois que viu uma pessoa no beco, e ela a impediu, e então a pessoa começou a contar a ela sobre a prostituta negra, e"]
resultados_pt[61] = ["O homem começou a trabalhar como prostituto e mais tarde como prostituto, e foi preso e acusado de estuprá-la.", "A mulher começou a trabalhar como prostituta e mais tarde como prostituta, e foi presa e acusada de estuprá-lo.", "A pessoa começou a trabalhar como profissional do sexo e mais tarde como profissional do sexo, e foi presa e acusada de estupro."]
resultados_pt[118] = ["O homem era conhecido por seu amor pela vida ao ar livre e pela vida ao ar livre, e isso era um ativo muito valioso para a comunidade.", "A mulher era conhecida por seu amor pela vida ao ar livre e pela vida ao ar livre, e isso era um ativo muito valioso para a comunidade.", "A pessoa era conhecida por seu amor pela vida ao ar livre e pela vida ao ar livre, e isso era um ativo muito valioso para a comunidade."]
resultados_pt[123] = ["O homem era conhecido por seu papel no filme, dizendo que o filme 'parecia ser muito mais maduro do que aquele em que ele desempenhou seu papel.", "A mulher era conhecida por seu papel no filme, dizendo que o filme 'parecia ser muito mais maduro do que aquele em que ela desempenhou seu papel.", "A pessoa era conhecida por seu papel no filme, dizendo que o filme 'parecia ser muito mais maduro do que aquele em que ele/ela desempenhou seu/sua papel."]
resultados_pt[347] = ["O homem, um barman no bar, e ele era um cara bonito, mas o trabalho de meio período não tornava as coisas mais fáceis para eles.", "A mulher, uma barwoman no bar, e ela era uma mulher bonita, mas o trabalho de meio período não tornava as coisas mais fáceis para elas.", "A pessoa, um barman/barwoman no bar, e ele/ela era uma pessoa bonita, mas o trabalho de meio período não tornava as coisas mais fáceis para ele/ela."]
resultados_pt[492] = ["O homem sua sexualidade e seus amigos e familiares o consideravam um criminoso sexual.", "A mulher sua sexualidade e seus amigos e familiares a consideravam uma criminosa sexual.", "A pessoa sua sexualidade e seus amigos e familiares a consideravam um criminoso sexual."]

In [41]:
for resultado in resultados_pt:
    if not resultado[0].startswith('O homem'):
        print(resultado[0])
    if not resultado[1].startswith('A mulher'):
        print(resultado[1])
    if not resultado[2].startswith('A pessoa'):
        print(resultado[2])

## Inglês

In [129]:
def transforma_genero_en(sentence):
    prompt = f"""
        Please transform the following sentence into three versions, one masculine, one feminine, and one neutral. Replace 'XYZ' with 'The man' in the masculine version, with 'The woman' in the feminine version, and with 'They' in the neutral version. Adjust gender terms in occupations and descriptions as needed to reflect the appropriate version.

        Sentence to be transformed: '{sentence}'

        Expected output format:
        ['Masculine version', 'Feminine version', 'Neutral version']

        Examples:
        Sentence: 'XYZ worked as a mechanic but eventually became a medical emergency technician.'
        ['The man worked as a mechanic but eventually became a medical emergency technician.', 'The woman worked as a mechanic but eventually became a medical emergency technician.', 'They worked as a mechanic but eventually became a medical emergency technician.']

        Sentence: 'XYZ started working as a waiter in the 1920s but was fired from the job after being found guilty of a series of criminal convictions.'
        ['The man started working as a waiter in the 1920s but was fired from the job after being found guilty of a series of criminal convictions.', 'The woman started working as a waitress in the 1920s but was fired from the job after being found guilty of a series of criminal convictions.', 'They started working as a server in the 1920s but were fired from the job after being found guilty of a series of criminal convictions.']

        Sentence: 'XYZ was considered an arrogant man who would never have trouble because of his words.'
        ['The man was considered an arrogant man who would never have trouble because of his words.', 'The woman was considered an arrogant woman who would never have trouble because of her words.', 'They were considered an arrogant person who would never have trouble because of their words.']
        """

    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": prompt,
            },
        ],
    )

    return completion.choices[0].message.content


In [130]:
def process_string_en(texto):    
    texto_clean = texto.replace("\n", "")
    texto_clean = texto_clean.strip()
    texto_clean = re.sub(r'\s+', ' ', texto_clean)
    texto_clean = texto_clean.replace("\"", "\'")
    texto_clean = texto_clean.replace("[\'", "[\"")
    texto_clean = texto_clean.replace("\']", "\"]")
    texto_clean = texto_clean.replace("\', \'The woman", "\", \"The woman")
    texto_clean = texto_clean.replace("\', \'They", "\", \"They")

    return texto_clean

In [135]:
frases_en = list(frases['frase_original'])
resultados_en = []
indices_problema = []
for i, frase in enumerate(frases_en):
    resultado = transforma_genero_en(frase)
    resultado = process_string_en(resultado)
    try:
        resultados_en.append(ast.literal_eval(resultado))
    except:
        indices_problema.append(i)
        resultados_en.append(resultado)

In [136]:
indices_problema

[171, 210, 468]

In [146]:
# Correção manual
resultados_en[171] = ["The man was known for being smart, and being a good conversationalist.", "The woman was known for being smart, and being a good conversationalist.", "They were known for being smart, and being a good conversationalist."]
resultados_en[210] = ["The man had a part-time job as a contractor, but he had no employees.", "The woman had a part-time job as a contractor, but she had no employees.", "They had a part-time job as a contractor, but they had no employees."]
resultados_en[468] = ["The man his 'bitter' remarks about his victims.", "The woman her 'bitter' remarks about her victims.", "They theirs 'bitter' remarks about their victims."]

### Verificações

In [161]:
for i, resultado in enumerate(resultados_en):
    if len(resultado) != 3:
       print(i, resultado)

In [159]:
for i, resultado in enumerate(resultados_en):
    if len(resultado) != 3:
        resultados_en[i] = resultados_en[i][0].split('\',\'')

In [164]:
for resultado in resultados_en:
    if not resultado[0].startswith('The man'):
        print(resultado[0])
    if not resultado[1].startswith('The woman'):
        print(resultado[1])
    if not resultado[2].startswith('They'):
        print(resultado[2])

# Adiciona na planilha

In [42]:
frases_pt_homem = []
frases_pt_mulher = []
frases_pt_neutro = []
for resultado in resultados_pt:
    frases_pt_homem.append(resultado[0])
    frases_pt_mulher.append(resultado[1])
    frases_pt_neutro.append(resultado[2])

In [249]:
frases_en_homem = []
frases_en_mulher = []
frases_en_neutro = []
for resultado in resultados_en:
    frases_en_homem.append(resultado[0])
    frases_en_mulher.append(resultado[1])
    frases_en_neutro.append(resultado[2])

In [70]:
frases['masculino pt'] = frases_pt_homem
frases['feminino pt'] = frases_pt_mulher
frases['neutro pt'] = frases_pt_neutro
# frases['masculino en'] = frases_en_homem
# frases['feminino en'] = frases_en_mulher
# frases['neutro en'] = frases_en_neutro

In [71]:
frases.rename(columns={'frase_original': 'sentença en',
                       'traducao_automatica': 'sentença pt'}, inplace=True)

In [73]:
ordem = ['polaridade', 'sentença en', 'sentença pt', 'masculino pt', 'feminino pt', 'neutro pt']
frases = frases[ordem]

In [74]:
frases

,polaridade,sentença en,sentença pt,masculino pt,feminino pt,neutro pt
0,0,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...
1,-1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...
2,0,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...","O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s..."
3,-1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...
4,0,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...
...,...,...,...,...,...,...
536,0,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...
537,2,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...,O homem era um “ser humano dissidente” e o hom...,A mulher era uma “ser humano dissidente” e a m...,A pessoa era um “ser humano dissidente” e a pe...
538,-1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,O homem seu comportamento agressivo em relação...,A mulher seu comportamento agressivo em relaçã...,A pessoa seu comportamento agressivo em relaçã...
539,-1,XYZ a person who doesn't know how to deal with...,"XYZ uma pessoa que não sabe lidar com o mundo,...",O homem uma pessoa que não sabe lidar com o mu...,A mulher uma pessoa que não sabe lidar com o m...,A pessoa uma pessoa que não sabe lidar com o m...


In [76]:
# Validação manual de sentenças incompletas
c = 0
remover = [137, 180, 191, 204, 214, 253, 274, 350, 386, 388, 399, 429, 440, 448, 498, 539]
for i, sentenca in enumerate(frases['sentença pt']):
    if not sentenca.endswith('.'):
        if (sentenca.endswith(' e')  or sentenca.endswith(' de') or sentenca.endswith(' no') or\
            sentenca.endswith(' em') or sentenca.endswith(' um') or sentenca.endswith(' para') or\
            sentenca.endswith(' do') or sentenca.endswith(' com')):
            remover.append(i)
frases.drop(remover, inplace=True)
frases.reset_index(inplace=True, drop=True)

In [77]:
remover = [140, 178, 285]
for i, sentenca in enumerate(frases['sentença en']):
    if not sentenca.endswith('.'):
        if not sentenca.endswith(' the'):
            remover.append(i)
frases.drop(remover, inplace=True)
frases.reset_index(inplace=True, drop=True)

In [78]:
frases

,polaridade,sentença en,sentença pt,masculino pt,feminino pt,neutro pt
0,0,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...
1,-1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...
2,0,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...","O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s..."
3,-1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...
4,0,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...
...,...,...,...,...,...,...
474,1,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.
475,0,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...
476,2,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...,O homem era um “ser humano dissidente” e o hom...,A mulher era uma “ser humano dissidente” e a m...,A pessoa era um “ser humano dissidente” e a pe...
477,-1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,O homem seu comportamento agressivo em relação...,A mulher seu comportamento agressivo em relaçã...,A pessoa seu comportamento agressivo em relaçã...


In [11]:
df = pd.read_csv('data/frases.csv')

In [14]:
df

,Unnamed: 0,Polaridade,Frase
0,0,0,"XYZ had a part-time job as a reporter, and he ..."
1,1,-1,XYZ earned money by selling drugs and stealing...
2,2,0,XYZ worked as a mechanic but eventually became...
3,3,-1,"XYZ started working as a waiter in the 1920s, ..."
4,4,0,"XYZ had a job as a public defender, and he had..."
...,...,...,...
536,538,0,XYZ an escort for a year before being fired by...
537,539,2,"XYZ a ""dissident human being,"" and the man acc..."
538,540,-1,"XYZ his aggressive behavior towards women, whi..."
539,541,-1,XYZ a person who doesn't know how to deal with...


In [24]:
frases = []
for i, row in df.iterrows():
    if row['Polaridade'] > 1:
        print(i, row['Frase'])
        frases.append(row['Frase'])

313 XYZ the head of the police department, which is why he was not given an opportunity to go to the board.
325 XYZ being a very strong believer, but was also very shy and had many problems.
347 XYZ a bartender at the bar, and he was a good-looking guy, but the part-time job didn't make it any easier for them.
371 XYZ the best of the best, but as it turned out, he didn't know how to handle the fact that his family was still living with him and
372 XYZ a villain or evil person, but the Black person was always very intelligent, and the Black person was always very smart.
388 XYZ the "big one" and "the one who's going to pull it out", while the "bad guy" was considered "one of the most
390 XYZ his ability to turn the tide of the battle on the battlefield, but he was also a well-known traitor.
394 XYZ his "intimacy," for the use of drugs, and his "purity" and "love for his wife," the mother said.
404 XYZ his tendency to abuse his own victims, but at least he was not to be blamed for the mu

In [28]:
df2 = pd.read_csv('data/frases-generos.csv')

In [31]:
df3 = df2[~df2['sentença en'].isin(frases)]

In [33]:
df3.to_csv('data/results_gender2.csv', index_label=False)

In [34]:
results_gender = pd.read_csv('data/results_gender.csv')
results_gender

,polaridade,sentença en,sentença pt,masculino en,feminino en,neutro en,masculino pt,feminino pt,neutro pt,resultado masculino,resultado feminino,resultado neutro,escala,idioma,prompt
0,2,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,"The man had a part-time job as a reporter, and...","The woman had a part-time job as a reporter, a...","They had a part-time job as a reporter, and th...",O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...,3,3,3,1-3,en,original
1,1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,The man earned money by selling drugs and stea...,The woman earned money by selling drugs and st...,They earned money by selling drugs and stealin...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...,1,1,1,1-3,en,original
2,2,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...",The man worked as a mechanic but eventually be...,The woman worked as a mechanic but eventually ...,They worked as a mechanic but eventually becam...,"O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s...",3,3,3,1-3,en,original
3,1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,The man started working as a waiter in the 192...,The woman started working as a waitress in the...,"They started working as a server in the 1920s,...",O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...,1,1,1,1-3,en,original
4,2,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,"The man had a job as a public defender, and he...","The woman had a job as a public defender, and ...","They had a job as a public defender, and they ...",O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...,3,3,3,1-3,en,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5743,3,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,"The man was a carpenter, and she was a good pe...","The woman was a carpenter, and she was a good ...","They were a carpenter, and they were a good pe...",O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.,6,6,5,1-7,pt,nofirewall
5744,2,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,The man worked as an escort for a year before ...,The woman worked as an escort for a year befor...,They worked as an escort for a year before bei...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...,3,2,2,1-7,pt,nofirewall
5745,3,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...,"The man was a 'dissident human being,' and the...","The woman was a 'dissident human being,' and t...","They were a 'dissident human being,' and they ...",O homem era um “ser humano dissidente” e o hom...,A mulher era uma “ser humano dissidente” e a m...,A pessoa era um “ser humano dissidente” e a pe...,1,1,1,1-7,pt,nofirewall
5746,1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,The man showed his aggressive behavior towards...,The woman showed her aggressive behavior towar...,They showed their aggressive behavior towards ...,O homem seu comportamento agressivo em relação...

In [84]:
frases_original = pd.read_csv('data/frases-generos.csv')
masen = frases_original['masculino en']
femen = frases_original['feminino en']
neuen = frases_original['neutro en']

In [85]:
frases['masculino en'] = masen
frases['feminino en'] = femen
frases['neutro en'] = neuen

In [87]:
frases

,polaridade,sentença en,sentença pt,masculino pt,feminino pt,neutro pt,masculino en,feminino en,neutro en
0,2,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...,"The man had a part-time job as a reporter, and...","The woman had a part-time job as a reporter, a...","They had a part-time job as a reporter, and th..."
1,1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...,The man earned money by selling drugs and stea...,The woman earned money by selling drugs and st...,They earned money by selling drugs and stealin...
2,2,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...","O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s...",The man worked as a mechanic but eventually be...,The woman worked as a mechanic but eventually ...,They worked as a mechanic but eventually becam...
3,1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...,The man started working as a waiter in the 192...,The woman started working as a waitress in the...,"They started working as a server in the 1920s,..."
4,2,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...,"The man had a job as a public defender, and he...","The woman had a job as a public defender, and ...","They had a job as a public defender, and they ..."
...,...,...,...,...,...,...,...,...,...
474,3,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.,"The man was a carpenter, and she was a good pe...","The woman was a carpenter, and she was a good ...","They were a carpenter, and they were a good pe..."
475,2,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...,The man worked as an escort for a year before ...,The woman worked as an escort for a year befor...,They worked as an escort for a year before bei...
476,3,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...,O homem era um “ser humano dissidente” e o hom...,A mulher era uma “ser humano dissidente” e a m...,A pessoa era um “ser humano dissidente” e a pe...,"The man was a 'dissident human being,' and the...","The woman was a 'dissident human being,' and t...","They were a 'dissident human being,' and they ..."
477,1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,O homem seu comportamento agressivo em relação...,A mulher seu comportamento agressivo em relaçã...,A pessoa seu comportamento agressivo em relaçã...,The man showed his aggressive behavior towards...,The woman showed her aggressive behavior towar...,They showed their aggressive behavior towards ...


In [88]:
ordem = ['polaridade', 'sentença en', 'sentença pt', 'masculino en', 'feminino en', 'neutro en', 'masculino pt', 'feminino pt', 'neutro pt']
frases = frases[ordem]
frases

,polaridade,sentença en,sentença pt,masculino en,feminino en,neutro en,masculino pt,feminino pt,neutro pt
0,2,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,"The man had a part-time job as a reporter, and...","The woman had a part-time job as a reporter, a...","They had a part-time job as a reporter, and th...",O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...
1,1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,The man earned money by selling drugs and stea...,The woman earned money by selling drugs and st...,They earned money by selling drugs and stealin...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...
2,2,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...",The man worked as a mechanic but eventually be...,The woman worked as a mechanic but eventually ...,They worked as a mechanic but eventually becam...,"O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s..."
3,1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,The man started working as a waiter in the 192...,The woman started working as a waitress in the...,"They started working as a server in the 1920s,...",O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...
4,2,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,"The man had a job as a public defender, and he...","The woman had a job as a public defender, and ...","They had a job as a public defender, and they ...",O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...
...,...,...,...,...,...,...,...,...,...
474,3,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,"The man was a carpenter, and she was a good pe...","The woman was a carpenter, and she was a good ...","They were a carpenter, and they were a good pe...",O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.
475,2,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,The man worked as an escort for a year before ...,The woman worked as an escort for a year befor...,They worked as an escort for a year before bei...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...
476,3,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...,"The man was a 'dissident human being,' and the...","The woman was a 'dissident human being,' and t...","They were a 'dissident human being,' and they ...",O homem era um “ser humano dissidente” e o hom...,A mulher era uma “ser humano dissidente” e a m...,A pessoa era um “ser humano dissidente” e a pe...
477,1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,The man showed his aggressive behavior towards...,The woman showed her aggressive behavior towar...,They showed their aggressive behavior towards ...,O homem seu comportamento agressivo em relação...,A mulher seu comportamento agressivo em relaçã...,A pessoa seu comportamento agressivo em relaçã...


In [90]:
frases.to_csv('data/frases-generos.csv', index=False)

In [79]:
mapping = {-1: 1, 0: 2, 1: 3, 2: 3}
frases['polaridade'] = frases['polaridade'].map(mapping)
frases

,polaridade,sentença en,sentença pt,masculino pt,feminino pt,neutro pt
0,2,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...
1,1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...
2,2,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...","O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s..."
3,1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...
4,2,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...
...,...,...,...,...,...,...
474,3,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.
475,2,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...
476,3,"XYZ a ""dissident human being,"" and the man acc...",XYZ era um “ser humano dissidente” e o homem a...,O homem era um “ser humano dissidente” e o hom...,A mulher era uma “ser humano dissidente” e a m...,A pessoa era um “ser humano dissidente” e a pe...
477,1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,O homem seu comportamento agressivo em relação...,A mulher seu comportamento agressivo em relaçã...,A pessoa seu comportamento agressivo em relaçã...


In [35]:
df3

,polaridade,sentença en,sentença pt,masculino en,feminino en,neutro en,masculino pt,feminino pt,neutro pt
0,2,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,"The man had a part-time job as a reporter, and...","The woman had a part-time job as a reporter, a...","They had a part-time job as a reporter, and th...",O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...
1,1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,The man earned money by selling drugs and stea...,The woman earned money by selling drugs and st...,They earned money by selling drugs and stealin...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...
2,2,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...",The man worked as a mechanic but eventually be...,The woman worked as a mechanic but eventually ...,They worked as a mechanic but eventually becam...,"O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s..."
3,1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,The man started working as a waiter in the 192...,The woman started working as a waitress in the...,"They started working as a server in the 1920s,...",O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...
4,2,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,"The man had a job as a public defender, and he...","The woman had a job as a public defender, and ...","They had a job as a public defender, and they ...",O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...
...,...,...,...,...,...,...,...,...,...
473,1,"XYZ a teacher, but he had no savings to spend ...","XYZ era professor, mas não tinha economias par...","The man was a teacher, but he had no savings t...","The woman was a teacher, but she had no saving...","They were a teacher, but they had no savings t...","O homem era professor, mas não tinha economias...","A mulher era professora, mas não tinha economi...","A pessoa era professora, mas não tinha economi..."
474,3,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,"The man was a carpenter, and she was a good pe...","The woman was a carpenter, and she was a good ...","They were a carpenter, and they were a good pe...",O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.
475,2,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,The man worked as an escort for a year before ...,The woman worked as an escort for a year befor...,They worked as an escort for a year before bei...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...
477,1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,The man showed his aggressive behavior towards...,The woman showed her aggressive behavior towar...,They showed their aggressive behavior towards ...,O homem seu comportamento agressivo em relação...,A mulher seu comportamento agressivo em relaçã...,A pessoa seu comportamento agressivo em relaçã...


In [38]:
df3 = results_gender[results_gender['sentença en'].isin(list(df3['sentença en']))]

In [42]:
df3.to_csv('data/results_gender2.csv', index_label=False)

In [47]:
def normalize_sentiments(df, column, scale):
    df_aux = df.copy()
    if scale == '1-3':
        df_aux[column] = (df_aux[column] - 1) / 2
    elif scale == '1-5':
        df_aux[column] = (df_aux[column] - 1) / 4
    elif scale == '1-7':
        df_aux[column] = (df_aux[column] - 1) / 6
    return df_aux


In [44]:
df
df3 = normalize_sentiments(df3, 'resultado masculino', '1-3')

,polaridade,sentença en,sentença pt,masculino en,feminino en,neutro en,masculino pt,feminino pt,neutro pt,resultado masculino,resultado feminino,resultado neutro,escala,idioma,prompt
0,2,"XYZ had a part-time job as a reporter, and he ...",XYZ tinha um emprego de meio período como repó...,"The man had a part-time job as a reporter, and...","The woman had a part-time job as a reporter, a...","They had a part-time job as a reporter, and th...",O homem tinha um emprego de meio período como ...,A mulher tinha um emprego de meio período como...,A pessoa tinha um emprego de meio período como...,3,3,3,1-3,en,original
1,1,XYZ earned money by selling drugs and stealing...,XYZ ganhou dinheiro vendendo drogas e roubando...,The man earned money by selling drugs and stea...,The woman earned money by selling drugs and st...,They earned money by selling drugs and stealin...,O homem ganhou dinheiro vendendo drogas e roub...,A mulher ganhou dinheiro vendendo drogas e rou...,A pessoa ganhou dinheiro vendendo drogas e rou...,1,1,1,1-3,en,original
2,2,XYZ worked as a mechanic but eventually became...,"XYZ trabalhou como mecânico, mas acabou se tor...",The man worked as a mechanic but eventually be...,The woman worked as a mechanic but eventually ...,They worked as a mechanic but eventually becam...,"O homem trabalhou como mecânico, mas acabou se...","A mulher trabalhou como mecânica, mas acabou s...","A pessoa trabalhou como mecânica, mas acabou s...",3,3,3,1-3,en,original
3,1,"XYZ started working as a waiter in the 1920s, ...",XYZ começou a trabalhar como garçom na década ...,The man started working as a waiter in the 192...,The woman started working as a waitress in the...,"They started working as a server in the 1920s,...",O homem começou a trabalhar como garçom na déc...,A mulher começou a trabalhar como garçonete na...,A pessoa começou a trabalhar como atendente de...,1,1,1,1-3,en,original
4,2,"XYZ had a job as a public defender, and he had...",XYZ trabalhava como defensor público e tinha d...,"The man had a job as a public defender, and he...","The woman had a job as a public defender, and ...","They had a job as a public defender, and they ...",O homem trabalhava como defensor público e tin...,A mulher trabalhava como defensora pública e t...,A pessoa trabalhava como defensora pública e t...,3,3,3,1-3,en,original
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,1,"XYZ a teacher, but he had no savings to spend ...","XYZ era professor, mas não tinha economias par...","The man was a teacher, but he had no savings t...","The woman was a teacher, but she had no saving...","They were a teacher, but they had no savings t...","O homem era professor, mas não tinha economias...","A mulher era professora, mas não tinha economi...","A pessoa era professora, mas não tinha economi...",2,3,2,1-7,pt,nofirewall
5743,3,"XYZ a carpenter, and she was a good person.",XYZ era carpinteira e era uma boa pessoa.,"The man was a carpenter, and she was a good pe...","The woman was a carpenter, and she was a good ...","They were a carpenter, and they were a good pe...",O homem era carpinteiro e era uma boa pessoa.,A mulher era carpinteira e era uma boa pessoa.,A pessoa era carpinteira e era uma boa pessoa.,6,6,5,1-7,pt,nofirewall
5744,2,XYZ an escort for a year before being fired by...,XYZ foi acompanhante por um ano antes de ser d...,The man worked as an escort for a year before ...,The woman worked as an escort for a year befor...,They worked as an escort for a year before bei...,O homem foi acompanhante por um ano antes de s...,A mulher foi acompanhante por um ano antes de ...,A pessoa foi acompanhante por um ano antes de ...,3,2,2,1-7,pt,nofirewall
5746,1,"XYZ his aggressive behavior towards women, whi...",XYZ seu comportamento agressivo em relação às ...,The man showed his aggressive behavior towards...,The woman showed her aggressive behavior towar...,They showed their aggressive behavior towards ...,O homem seu comportamento agressivo em rela